In [ ]:
import math
from typing import List
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Inputs

#accuracies = [0.9363,0.9991,0.8213,0.997]
#training_times = [1662,584,694,277]
#model_names =["CNN 1D-Time","CNN 1D-Frequency","CNN 2D-Time","CNN 2D-Frequency"]


accuracies = [0.8213,0.997]
training_times = [694,277]
model_names =["CNN 2D-Time","CNN 2D-Frequency"]


#accuracies = [0.3,0.31,0.78,0.43,0.72,0.45,0.98,0.98]
#training_times = [796,14226,32,74,31,64,552,383]
#model_names =["RF","SVM","RF + Feat. Ext.","SVM + Feat. Ext.","RF + Feat. Ext. + PCA","SVM + Feat. Ext. + PCA","RF + FFT","SVM + FFT"]


#accuracies = [0.3,0.31,0.78,0.43,0.98,0.98]
#training_times = [796,14226,32,74,552,383]
#model_names =["RF","SVM","RF + FeatExt","SVM + FeatExt","RF + FFT","SVM + FFT"]


alphas = (0.2, 0.5, 0.7)


if not (len(model_names) == len(accuracies) == len(training_times)):
    raise ValueError("model_names, accuracies and training_times should have the same dimension.")
if len(model_names) == 0:
    raise ValueError("Gives at leat one model.")

In [ ]:
# Compute tn (normalized ) model training time of each model

def normalize_time_log(times: List[float]) -> np.ndarray:
    """
    t_n in [0,1] via escala log:
      t_n = (log t - log t_min) / (log t_max - log t_min)
    Requer t > 0. Se t_min == t_max devolve zeros.
    """
    times = np.asarray(times, dtype=float)
    if np.any(times <= 0):
        raise ValueError("Todos os tempos devem ser > 0 para normalização log.")
    t_min, t_max = times.min(), times.max()
    if t_min == t_max:
        return np.zeros_like(times)
    tn = (np.log(times) - np.log(t_min)) / (np.log(t_max) - np.log(t_min))
    return np.clip(tn, 0.0, 1.0)

tn = normalize_time_log(training_times)
print(tn)

In [ ]:
#Compute S (Speed of each model)

def compute_speed_from_tn(tn: np.ndarray) -> np.ndarray:
    """Speed normalizada = 1 - t_n."""
    return 1.0 - tn

speed = compute_speed_from_tn(tn)
print(speed)

In [ ]:
#Compute GPI of each model

def compute_gpi(acc: List[float], speed: np.ndarray, alphas) -> pd.DataFrame:
    """
    GPI_alpha = alpha*accuracy + (1-alpha)*speed para cada alpha.
    acc deve estar em [0,1].
    """
    acc = np.asarray(acc, dtype=float)
    if np.any((acc < 0) | (acc > 1)):
        raise ValueError("Todas as accuracies devem estar em [0,1].")
    if acc.shape != speed.shape:
        raise ValueError("accuracy e speed devem ter o mesmo comprimento.")
    data = {}
    for a in alphas:
        if not (0.0 <= a <= 1.0):
            raise ValueError("Cada alpha deve estar em [0,1].")
        data[f"GPI_alpha_{a}"] = a * acc + (1 - a) * speed
    return pd.DataFrame(data)

gpi_df = compute_gpi(accuracies, speed, alphas=alphas)


#Resume table
res = pd.DataFrame({
    "model": model_names,
    "accuracy": accuracies,
    "time_s": training_times,
    "t_n": tn,
    "Speed": speed
}).join(gpi_df)


first_gpi_col = gpi_df.columns[0]
res_sorted = res.sort_values(by=first_gpi_col, ascending=False).reset_index(drop=True)

res_sorted

In [ ]:
#Plot (bar chart) the GPI of each model to different values of alpha

x = np.arange(len(model_names))
width = 0.8 / len(gpi_df.columns)

plt.figure(figsize=(10, 6))
for i, col in enumerate(gpi_df.columns):
    offsets = x + (i - (len(gpi_df.columns)-1)/2) * width
    plt.bar(offsets, gpi_df[col].values, width=width, label=col)

plt.xticks(x, model_names, rotation=25, ha="right")
plt.yticks(np.arange(0, 1.1, 0.1))
plt.ylabel("GPI")
plt.title("Global Performance Index (GPI) for each CNN model - Dataset I")
plt.ylim(0, 1.01)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
#Plot (line chart) the GPI of each model to different values of alpha

plt.figure(figsize=(10, 6))


x = np.arange(len(model_names))


alpha_values = [float(c.split("_")[-1]) for c in gpi_df.columns]

for j, col in enumerate(gpi_df.columns):
    plt.plot(
        x,
        gpi_df[col].values,
        marker="o",
        label=f"α = {alpha_values[j]}"
    )

plt.xticks(x, model_names)
#plt.xticks(x, model_names, rotation=25, ha="right")
plt.yticks(np.arange(0, 1.1, 0.1))
plt.ylabel("GPI")
plt.title("Global Performance Index (GPI) for RF and SVM models - Dataset I")
plt.ylim(0, 1.01)
plt.legend(loc='upper left')
plt.grid(axis='x', linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()
